<h1 style="text-align:center">
    Pump it Up: Data Mining the Water Table
</h1>

# Import packages

In [1]:
import pickle
import pandas as pd 
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

# Data load

In [2]:
# Target 
target = pd.read_csv("train_target.csv")

# Target values replace 
target = target.replace({'status_group': {'functional' : 1, 
                                          'non functional' : 0, 
                                          'functional needs repair' : 2}})

# Values 
values = pd.read_csv("train_values.csv")

test = pd.read_csv("test_values.csv")

# Values and target merge  
data = values.merge(target, on = 'id')
data.set_index(['id'], inplace = True)

ID = test['id']
test.set_index(['id'], inplace = True)

# EDA

In [3]:
data.describe()

,amount_tsh,gps_height,longitude,latitude,num_private,region_code,district_code,population,construction_year,status_group
count,59400.000000,59400.000000,59400.000000,5.940000e+04,59400.000000,59400.000000,59400.000000,59400.000000,59400.000000,59400.000000
mean,317.650385,668.297239,34.077427,-5.706033e+00,0.474141,15.297003,5.629747,179.909983,1300.652475,0.688434
std,2997.574558,693.116350,6.567432,2.946019e+00,12.236230,17.587406,9.633649,471.482176,951.620547,0.599877
min,0.000000,-90.000000,0.000000,-1.164944e+01,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,33.090347,-8.540621e+00,0.000000,5.000000,2.000000,0.000000,0.000000,0.000000
50%,0.000000,369.000000,34.908743,-5.021597e+00,0.000000,12.000000,3.000000,25.000000,1986.000000,1.000000
75%,20.000000,1319.250000,37.178387,-3.326156e+00,0.000000,17.000000,5.000000,215.000000,2004.000000,1.000000
max,350000.000000,2770.000000,40.345193,-2.000000e-08,1776.000000,99.000000,80.000000,30500.000000,2013.000000,2.000000


In [4]:
test.describe()

,amount_tsh,gps_height,longitude,latitude,num_private,region_code,district_code,population,construction_year
count,14850.000000,14850.000000,14850.000000,1.485000e+04,14850.000000,14850.000000,14850.000000,14850.000000,14850.000000
mean,322.826983,655.147609,34.061605,-5.684724e+00,0.415084,15.139057,5.626397,184.114209,1289.708350
std,2510.968644,691.261185,6.593034,2.940803e+00,8.167910,17.191329,9.673842,469.499332,955.241087
min,0.000000,-57.000000,0.000000,-1.156459e+01,0.000000,1.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,33.069455,-8.443970e+00,0.000000,5.000000,2.000000,0.000000,0.000000
50%,0.000000,344.000000,34.901215,-5.049750e+00,0.000000,12.000000,3.000000,20.000000,1986.000000
75%,25.000000,1308.000000,37.196594,-3.320594e+00,0.000000,17.000000,5.000000,220.000000,2004.000000
max,200000.000000,2777.000000,40.325016,-2.000000e-08,669.000000,99.000000,80.000000,11469.000000,2013.000000


In [5]:
test.isna().sum()

amount_tsh                  0
date_recorded               0
funder                    869
gps_height                  0
installer                 877
longitude                   0
latitude                    0
wpt_name                    0
num_private                 0
basin                       0
subvillage                 99
region                      0
region_code                 0
district_code               0
lga                         0
ward                        0
population                  0
public_meeting            821
recorded_by                 0
scheme_management         969
scheme_name              7092
permit                    737
construction_year           0
extraction_type             0
extraction_type_group       0
extraction_type_class       0
management                  0
management_group            0
payment                     0
payment_type                0
water_quality               0
quality_group               0
quantity                    0
quantity_g

In [6]:
data.dtypes

amount_tsh               float64
date_recorded             object
funder                    object
gps_height                 int64
installer                 object
longitude                float64
latitude                 float64
wpt_name                  object
num_private                int64
basin                     object
subvillage                object
region                    object
region_code                int64
district_code              int64
lga                       object
ward                      object
population                 int64
public_meeting            object
recorded_by               object
scheme_management         object
scheme_name               object
permit                    object
construction_year          int64
extraction_type           object
extraction_type_group     object
extraction_type_class     object
management                object
management_group          object
payment                   object
payment_type              object
water_qual

In [7]:
data.recorded_by.value_counts()

GeoData Consultants Ltd    59400
Name: recorded_by, dtype: int64

In [8]:
for code in data.extraction_type_group.unique():
    print(code + ': ' + str(data[data['extraction_type_group'] == code]['extraction_type'].unique()))

gravity: ['gravity']
submersible: ['submersible' 'ksb']
swn 80: ['swn 80']
nira/tanira: ['nira/tanira']
india mark ii: ['india mark ii']
other: ['other']
mono: ['mono']
wind-powered: ['windmill']
afridev: ['afridev']
rope pump: ['other - rope pump']
india mark iii: ['india mark iii']
other handpump: ['other - swn 81' 'other - play pump' 'walimi' 'other - mkulima/shinyanga']
other motorpump: ['cemo' 'climax']


In [9]:
for code in data.extraction_type_class.unique():
    print(code + ': ' + str(data[data['extraction_type_class'] == code]['extraction_type_group'].unique()))

gravity: ['gravity']
submersible: ['submersible']
handpump: ['swn 80' 'nira/tanira' 'india mark ii' 'afridev' 'india mark iii'
 'other handpump']
other: ['other']
motorpump: ['mono' 'other motorpump']
wind-powered: ['wind-powered']
rope pump: ['rope pump']


In [10]:
for code in data.management_group.unique():
    print(code + ': ' + str(data[data['management_group'] == code]['management'].unique()))

user-group: ['vwc' 'wug' 'water board' 'wua']
other: ['other' 'other - school']
commercial: ['private operator' 'company' 'water authority' 'trust']
parastatal: ['parastatal']
unknown: ['unknown']


In [11]:
for code in data.payment_type.unique():
    print(code + ': ' + str(data[data['payment_type'] == code]['payment'].unique()))

annually: ['pay annually']
never pay: ['never pay']
per bucket: ['pay per bucket']
unknown: ['unknown']
on failure: ['pay when scheme fails']
other: ['other']
monthly: ['pay monthly']


In [12]:
for code in data.quality_group.unique():
    print(code + ': ' + str(data[data['quality_group'] == code]['water_quality'].unique()))

good: ['soft']
salty: ['salty' 'salty abandoned']
milky: ['milky']
unknown: ['unknown']
fluoride: ['fluoride' 'fluoride abandoned']
colored: ['coloured']


In [13]:
for code in data.quantity_group.unique():
    print(code + ': ' + str(data[data['quantity_group'] == code]['quantity'].unique()))

enough: ['enough']
insufficient: ['insufficient']
dry: ['dry']
seasonal: ['seasonal']
unknown: ['unknown']


In [14]:
for code in data.source_type.unique():
    print(code + ': ' + str(data[data['source_type'] == code]['source'].unique()))

spring: ['spring']
rainwater harvesting: ['rainwater harvesting']
dam: ['dam']
borehole: ['machine dbh' 'hand dtw']
other: ['other' 'unknown']
shallow well: ['shallow well']
river/lake: ['river' 'lake']


In [15]:
for code in data.source_class.unique():
    print(code + ': ' + str(data[data['source_class'] == code]['source_type'].unique()))

groundwater: ['spring' 'borehole' 'shallow well']
surface: ['rainwater harvesting' 'dam' 'river/lake']
unknown: ['other']


In [16]:
for code in data.waterpoint_type_group.unique():
    print(code + ': ' + str(data[data['waterpoint_type_group'] == code]['waterpoint_type'].unique()))

communal standpipe: ['communal standpipe' 'communal standpipe multiple']
hand pump: ['hand pump']
other: ['other']
improved spring: ['improved spring']
cattle trough: ['cattle trough']
dam: ['dam']


# Feature Engineering

In [17]:
data['funder'] = data['funder'].fillna('Missing')
test['funder'] = test['funder'].fillna('Missing')

chars = []
for fund in data.funder:
    if fund not in chars:
        chars.append(fund)

for fun in test.funder:
        if fun not in chars:
            chars.append(fun)
            
equi = {}
num = 1 
for char in chars: 
    equi[char] = num
    num += 1
    
funders = []
for fund in data.funder:
    word = str(equi[fund])
    funders.append(word)
    
fundors = []
for fund in test.funder:
    word = str(equi[fund])
    fundors.append(word)
    
data['funder'] = pd.Series([int(num) for num in funders],index = data.index).astype('int')
test['funder'] = pd.Series([int(num) for num in fundors],index = test.index).astype('int')

In [18]:
data['y_recorded'] = [int(anno.split('-')[0]) for anno in data['date_recorded']]
data['m_recorded'] = [int(mes.split('-')[1]) for mes in data['date_recorded']]
data['d_recorded'] = [int(dia.split('-')[2]) for dia in data['date_recorded']]

test['y_recorded'] = [int(anno.split('-')[0]) for anno in test['date_recorded']]
test['m_recorded'] = [int(mes.split('-')[1]) for mes in test['date_recorded']]
test['d_recorded'] = [int(dia.split('-')[2]) for dia in test['date_recorded']]

y = data['status_group']

to_drop = ['date_recorded', 'recorded_by', 'extraction_type_group', 'extraction_type_class', 'management_group', 'payment_type',
           'quality_group', 'quantity_group', 'source_type', 'source_class', 'waterpoint_type_group', 'status_group']

to_drop = ['installer', 'wpt_name', 'scheme_management','subvillage', 'lga', 'ward', 'basin', 'scheme_name', 'payment', 
           'management', 'waterpoint_type_group'] + to_drop

data.drop(columns = to_drop, inplace =True)
test.drop(columns = to_drop[:-1], inplace =True)

In [19]:
# One Hot Encoding with get_dummy 
data = pd.get_dummies(data)
test = pd.get_dummies(test)

In [20]:
data.drop(columns = 'extraction_type_other - mkulima/shinyanga', inplace =True)

## Grid

In [27]:
grid_param = {
    'n_estimators': [100, 300, 500, 800, 1000],
    'criterion': ['gini', 'entropy'],
    'bootstrap': [True, False]
}

In [28]:
model_grid = GridSearchCV(estimator = RandomForestClassifier(),
                     param_grid = grid_param,
                     scoring = 'accuracy',
                     n_jobs = -1,
                     cv = 5)

In [29]:
model_grid.fit(data, y)

GridSearchCV(cv=5, estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'bootstrap': [True, False],
                         'criterion': ['gini', 'entropy'],
                         'n_estimators': [100, 300, 500, 800, 1000]},
             scoring='accuracy')

In [30]:
print(model_grid.best_params_)

{'bootstrap': True, 'criterion': 'entropy', 'n_estimators': 500}


In [31]:
print(model_grid.best_score_)

0.8075084175084175


# Model Training

In [21]:
model = RandomForestClassifier(bootstrap =  True, criterion = 'entropy', n_estimators = 800)
model.fit(data, y)

RandomForestClassifier(criterion='entropy', n_estimators=800)

# Prediciones

In [23]:
final_preds = model.predict(test)

In [24]:
preds = pd.concat([ID, pd.Series(final_preds, name = 'status_group')], axis=1)

# Replace target values - there are three classes
preds = preds.replace({'status_group': {1 : 'functional', 
                                        0 : 'non functional', 
                                        2 : 'functional needs repair'}})
preds

,id,status_group
0,50785,non functional
1,51630,functional
2,17168,functional
3,45559,non functional
4,49871,functional
...,...,...
14845,39307,non functional
14846,18990,functional
14847,28749,functional
14848,33492,functional


In [25]:
preds.to_csv('preds.csv', index = False)  

# Save model

In [27]:
pkl_filename = "Model.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(model, file)